**Research Question 2**

How often is a Q1 fracture associated with an ECTP on the same layer?

## General Process
1. load files
2. Optional: Find pits with properties of interest
3. Create a list of dictionaries of properties of interest
4. Create a dataframe from the list of dictionaries

*Question: In the case of multiple CT and ECT results, which results should be used?*
* Do any tests


* Do any ECTs and CTs have P and Q1 on the same layer?
    If yes, then correlation


* Are ECTP and Q1 failure on the same layer?
* If multiple CTs, are they all the same shear quality?


*STEPS*
1. find pits with ECTP and CT with shear quality results
2. Find tests with failure on the same layer
3. If multiple of either test, select test with lowest fracture character

*FIRST QUESTION*
* Of pits that have ECTP results and CT results with Shear quality. When failure occurs on the same layer, how often (pie chart and/or percentage table) is the fracture character of the CT Q1?








In [1]:
# Load files
import os
import pandas as pd
from caaml_parser import caaml_parser

folder_path = "snowpits_200_MT" # The latest 200 snow pits from MT

# Create a list of all CAAML files in the folder
caaml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')] # List of all CAAML files in the folder

pit1
SnowPit: 
 caamlVersion: {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3} 
 pitID: 66387 
 Date: 2024-11-06 
 User:
    OperationID: None
    OperationName: None
    Professional: None
    ContactPersonID: None
    ContactPersonName: None
 Location:
    Latitude: 45.819852
    Longitude: -110.928698
    Elevation: ['2532', 'm']
    Aspect: ['E', None]
    SlopeAngle: ['31', 'deg']
    Country: US
    Region: MT
 Snow Profile: 
    measurementDirection: top down
    profileDepth: [68.0, 'cm']
    hS: [68.0, 'cm']
    surfCond: 
	 windLoading: None
	 penetrationFoot: None
	 penetrationSki: None
    Layers:
    Layer 1: 
	 depthTop: ['0', 'cm']
	 thickness: ['8', 'cm']
	 grainFormPrimary: DFbk
	 hardness: F
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: DF
    Layer 2: 
	 depthTop: ['8', 'cm']
	 thickness: ['39', 'cm']
	 grainFormPrimary: DFbk
	 hardness: F+
	 wetness: None
	 layerOfConcern: None
	 grainFormPrimary_Class: DF
    Layer 3: 
	 depthTop: ['47', 'cm']
	

In [2]:
# Find pits with CT and ECT results

pits_with_CT_ECT = []

for file in caaml_files:
    file_path = folder_path + '/' + file
    pit = caaml_parser(file_path)

    if len(pit.stabilityTests.CT) > 0 and len(pit.stabilityTests.ECT) > 0:
        pits_with_CT_ECT.append(pit)


print(len(pits_with_CT_ECT)) ## How many pits have CT and ECT results?

55


In [14]:
# ECT and CT results

results_list=[]
results_dict={}

for pit in pits_with_CT_ECT:

    ECTs = pit.stabilityTests.ECT
    CTs = pit.stabilityTests.CT

    for ect in ECTs:
        for ct in CTs:
            if(ect.depthTop == ct.depthTop): # If failure on the same level
                results_dict ={
                    'PitID': pit.pitID,
                    'ECT Score': ect.testScore,
                    'ECT Prop': ect.propagation,
                    'CT Shear Qual': ct.shearQuality
                }

    results_list.append(results_dict)

df = pd.DataFrame(results_list)

print(df)


    PitID ECT Score CT Shear Qual
0     NaN       NaN           NaN
1   66408    ECTN23            Q2
2   66408    ECTN23            Q2
3   66504    ECTN18            Q1
4   66506    ECTN16            Q2
5   66670    ECTP15           BRK
6   66672    ECTP12           BRK
7   66731    ECTN21            Q1
8   66733    ECTN15            Q2
9   66741    ECTN30            Q2
10  66781    ECTN27            Q2
11  66781    ECTN27            Q2
12  66781    ECTN27            Q2
13  66781    ECTN27            Q2
14  66932    ECTP30            Q2
15  66957    ECTN29            Q2
16  66999    ECTN14            Q1
17  67018    ECTN30            Q3
18  67018    ECTN30            Q3
19  67018    ECTN30            Q3
20  67135      None          None
21  67135      None          None
22  67217    ECTN28            Q2
23  67408    ECTN13            SC
24  67408    ECTN13            SC
25  67563    ECTP21            Q1
26  67563    ECTP21            Q1
27  67634    ECTN12            RP
28  67634    E